# Inference

This Notebook can transform T2w sagittal nii.gz files into CT files.

[![DOI](https://zenodo.org/badge/605981537.svg)](https://zenodo.org/badge/latestdoi/605981537)

We provide our weight for DDIM image under 'logs_diffusion3D/Diffusion_3D_T2w_CT_spine_sag/version_0/T2_to_CT_iso_diffusion_img.pt' in the archived version https://zenodo.org/record/8221159 or under this mirror: [Drive-link](https://studentpartnersde-my.sharepoint.com/:f:/g/personal/robert_graf_studentpartners_de/El5c-wZ7TXZJjpo8snSNAAsBpyiAlM4vzaBO5dr3y79fAw?e=O3Xaq3)

In [ ]:
from math import ceil, floor
from pathlib import Path

import numpy as np
import torch
import torch.nn.functional as F
import yaml
from torch import Tensor
from TPTBox import NII

import utils.arguments as arguments
from pl_models.diffusion.diffusion import Diffusion

model_path = "logs_diffusion3D/Diffusion_3D_T2w_CT_spine_sag/version_0/T2_to_CT_iso_diffusion_img.pt"

with open(str(model_path).replace(".pt", ".yaml")) as file:
    opt = arguments.Diffusion_Option()
    opt.__dict__ = yaml.full_load(file)
    opt.conditional_label_size = 0
    opt.conditional_dimensions = 4
    in_channel = opt.in_channel  # type: ignore
diffusion = Diffusion(opt, channel=in_channel)
dic = torch.load(model_path)
diffusion.load_state_dict(dic, strict=True)

mri_path = "data/spine/sub-111168222_sequ-402_part-inphase_dixon.nii.gz"

In [ ]:
from utils.preprocessing import run_model, padded_shape


def prepare_nii(mri_path):
    nii = NII.load(mri_path, False, 0)
    nii_iso = nii.rescale((1, 1, 1), verbose=True).reorient(("R", "I", "P"))
    nii_iso /= nii_iso.max()
    nii_iso.clamp_(min=0)
    nii_iso = nii_iso * 2 - 1
    target_shape = padded_shape(nii_iso.shape)
    nii_iso = nii_iso.pad_to(target_shape)

    arr = Tensor(nii_iso.get_array().astype(float))
    # arr, padding = pad_size3D(arr)
    return arr, 0, nii, nii_iso

In [ ]:
def translate(mri_path, save=True, use_cpu=False):
    arr, padding, nii, nii_iso = prepare_nii(mri_path)
    if torch.cuda.is_available() and not use_cpu:
        # If you run out of memory use max_shape= (arr.shape[-1]*arr.shape[-2]*arr.shape[-3])//2
        ct_arr = run_model(diffusion, conditional=arr, gpu=True, eta=1, w=0, steps=25, depth=0)
    else:
        print("!!! Fall back to less steps on CPU instead of 25 GPU. !!!")
        ct_arr = run_model(diffusion, conditional=arr, gpu=False, eta=1, w=0, steps=5, depth=0)
    ct_nii_iso = nii_iso.set_array(ct_arr.numpy())
    ct_nii: NII = ct_nii_iso.set_dtype_("smallest_int").resample_from_to(nii)
    # revert_iso_to_original(ct_nii_iso, nii, padding)
    if "_" in mri_path:
        ct_path = mri_path.rsplit("_", maxsplit=1)[0] + "_desc-translated_ct.nii.gz"
        ct_pathi = mri_path.rsplit("_", maxsplit=1)[0] + "_desc-translated-iso_ct.nii.gz"
    else:
        ct_path = mri_path.rsplit(".", maxsplit=2)[0] + "_desc-translated_ct.nii.gz"
        ct_pathi = mri_path.rsplit(".", maxsplit=1)[0] + "_desc-translated-iso_ct.nii.gz"
    if save:
        ct_nii.set_dtype_("smallest_int").save(ct_path)
        ct_nii_iso.set_dtype_("smallest_int").save(ct_pathi)
    return ct_nii, ct_nii_iso

In [ ]:
mri_path = "data/spine/sub-111168222_sequ-402_part-inphase_dixon.nii.gz"
ct_nii, ct_nii_iso = translate(mri_path, save=True, use_cpu=False)

print(ct_nii)
print(ct_nii_iso)
print("You can now open the nii.gz")

In [ ]:
mri_path = "data/spine/sub-111168222_sequ-401_part-inphase_dixon.nii.gz"
translate(mri_path, save=True)
mri_path = "data/spine/sub-111168222_sequ-403_part-inphase_dixon.nii.gz"
translate(mri_path, save=True)